# S.VI.D, Fig.8: Evaluation with Larger Synthetic Task Sets

This notebook describes how to recreate the results shown in Figure 8 of Section VI.D: "Evaluation with Larger Synthetic Task Sets."

Source and header files include:
* `harmonic.h`: Provides the `Harmonic_Elastic` class, which encapsulates generating the complete set of projected harmonic intervals, and the corresponding lookup table, for a set of tasks. 
* `task.h`: Provides the `Task` data structure, which encapsulates the elastic task parameters. 
* `synthetic_tasksets.cc`: A wrapper to randomly generate task sets and collect data.
* `harmonic.cc`: Implements functionality for the `Harmonic_Elastic` class, including all algorithms listed in the paper.
* `task.cc`: Implements an information printing function for the `Task` structure.

## To Build

In [ ]:
%%sh
cd ..
make synthetic

## Generate Results

We have already provided the results data in the file `synthetic_results/synthetic_results.txt`. This can be reproduced by running the following:

In [ ]:
%%sh
./synthetic 1000 > synthetic_results/synthetic_results.txt

Output column values are:
* Number of tasks
* Task set index for that number (1--1000)
* Does a feasible harmonic period assignment exist (0/1)
* The number of projected harmonic zones into the last interval
* Execution time to run Algorithm 1 ($\mu$s) 

The remaining columns are only generated if harmonic periods were found.
* Number of projected harmonic intervals
* Number of intersecting regions generated in lookup table
* Execution time to generate the intervals and lookup table ($\mu$s)
* Execution time to search the lookup table and assign periods (ns)
* Execution time to use naive enumeration over all projected harmonic intervals to get the best for a given utilization, then adjust periods ($\mu$s)

## Plot and Analyze Results

The following steps will produce the plots in the paper.

In [ ]:
# Import Packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# Define Style for Plotting

# Use the fivethirtyeight style
plt.style.use('fivethirtyeight')

# Define a palette
colors = sns.color_palette("deep", 5)

In [ ]:
# Import Data

df = pd.read_table("synthetic_results.txt", delimiter=' ', header=None, )
df.fillna(0, inplace=True)
data = df.values[:,0:10]

max_tasks = 50
n_tasks = np.arange(5,max_tasks+1)


### The Harmonic Period Problem

We first evaluate the proposed approach to the harmonic period problem described in Section IV.B. For each number of tasks, we count how many of the 1000 corresponding sets had a feasible harmonic period assignment, how long it took to find such an assignment, and how many harmonic zones were projected onto the last interval.

#### Figure 8a

The following code plots Figure 8a, indicating for each size task set (from 5--50) how many of the 1000 generated had a feasible harmonic period selection.

In [ ]:
harmonic = data[:,2].reshape(max_tasks-4,1000)
harmonic = np.sum(harmonic, axis=1)

font = {'size'   : 13}
plt.rc('font', **font)

fig, ax1 = plt.subplots(figsize=(4.6, 2.8))

ax1.plot(n_tasks, harmonic, "-", color=colors[0], linewidth=2)
ax1.set_ylim(-50,1050)
ax1.set_xticks(np.arange(5,max_tasks+1,5))

ax1.set_xlabel("# Tasks")
ylabel = ax1.set_ylabel("# Harmonic Assign.")
label_position = ylabel.get_position()
ylabel.set_position((label_position[0], label_position[1] - 0.1))

plt.tight_layout()
name = "synthetic_num_harmonic"
plt.savefig(f"{name}.eps",bbox_inches='tight')
plt.savefig(f"{name}.png",bbox_inches='tight')

In [ ]:
# Generate Data for Fig 8b, 8c, 8d

#Maximum number of harmonic zones projected onto the last interval
n_projections = data[:,3].reshape(max_tasks-4,1000)
n_projections = np.max(n_projections, axis=1)

#Time to run Algorithm 1
harmonic_times = data[:,4].reshape(max_tasks-4,1000)
harmonic_times = np.max(harmonic_times, axis=1)

#Maximum number of projected harmonic intervals
n_phis = data[:,5].reshape(max_tasks-4,1000)
n_phis = np.max(n_phis, axis=1)

#Maximum number of regions in lookup table
n_regions = data[:,6].reshape(max_tasks-4,1000)
n_regions = np.max(n_regions, axis=1)

#Maximum time to generate lookup table
elastic_times_generate = data[:,7].reshape(max_tasks-4,1000)
elastic_times_generate = np.max(elastic_times_generate, axis=1)

#Maximum time to use lookup table
elastic_times_fast = data[:,8].reshape(max_tasks-4,1000) #ns
elastic_times_fast = np.max(elastic_times_fast, axis=1)

#Maximum time to iterate harmonic intervals (naive method)
elastic_times_naive = data[:,9].reshape(max_tasks-4,1000)
elastic_times_naive = np.max(elastic_times_naive, axis=1)

#Times collected in microseconds, use unit conversions where appropriate
ms = 1000
s = 1000000

#### Figure 8b

The following code generates Figure 8b. It both plots the maximum number of harmonic zones projected onto the last task's period interval for each size, as well as the maximum execution time of Algorithm 1 in Section IV.B.

In [ ]:
font = {'size'   : 13}
plt.rc('font', **font)

fig, ax1 = plt.subplots(figsize=(4.6, 2.8))

ax1.set_xlabel("# Tasks")
ax1.set_xticks(np.arange(5,max_tasks+1,5))

color = colors[0]
ax1.set_ylabel("Max Zones", color=color)
ax1.plot(n_tasks, n_projections, "-", color=color, linewidth=2)
ax1.set_ylim(0,np.max(n_projections)*1.05)

ax2 = ax1.twinx()

color = colors[1]
ax2.set_ylabel("Max Time (ms)", color=color)
ax2.plot(n_tasks, harmonic_times/ms, "--", color=color, linewidth=2)
ax2.set_ylim(0,np.max(harmonic_times/ms)*1.05)

plt.tight_layout()
name = "synthetic_projected_harmonic_zones"
plt.savefig(f"{name}.eps",bbox_inches='tight')
plt.savefig(f"{name}.png",bbox_inches='tight')

#### Maximum execution time

We also obtain the maximum execution time for Algorithm 1

In [ ]:

print(f"Maximum execution time of Algorithm 1: {np.max(harmonic_times)/ms} ms")

### The Ordered Harmonic Elastic Problem

We then evaluate both the naive and efficient approaches to the ordered harmonic elastic problem described in Section V. For those task sets where a harmonic assignment is possible, we count how many projected harmonic intervals (PHIs) are found, how long it takes to generate the lookup table (LUT), and how many entries it has. We also compare the time it takes to iterate over all PHIs to find the optimal period assignment, versus performing binary search. 

#### Figure 8c

The following code generates Figure 8c. It plots the maximum number of PHIs, the time to generate the lookup table, and the time to perform the naive search for the best PHI for a given utilization via enumeration.

In [ ]:
font = {'size'   : 13}
plt.rc('font', **font)

fig, ax1 = plt.subplots(figsize=(4.6, 2.8))

ax1.set_xlabel("# Tasks")
ax1.set_xticks(np.arange(5,max_tasks+1,5))

color = colors[0]
ax1.set_ylabel("Max PHIs", color=color)
ax1.plot(n_tasks, n_phis, "-", color=color, linewidth=2)
ax1.set_ylim(0,np.max(n_phis)*1.05)

ax2 = ax1.twinx()

color = colors[1]
ax2.set_ylabel("Max Time", color=color)
ax2.plot(n_tasks, elastic_times_generate/s, "--", color=color, label="Gen LUT (s)", linewidth=2)
ax2.plot(n_tasks, elastic_times_naive/ms, "-.", color='green', label="Naive Search (ms)", linewidth=2)
ax2.set_ylim(0,np.max(elastic_times_generate/s)*1.05)

plt.legend(loc='upper left', framealpha=1.0)

plt.tight_layout()
name = "synthetic_projected_harmonic_intervals"
plt.savefig(f"{name}.eps",bbox_inches='tight')
plt.savefig(f"{name}.png",bbox_inches='tight')

#### Figure 8d

The following code generates Figure 8d. It plots the maximum number of PHIs, the time to generate the lookup table, and the time to perform the naive search for the best PHI for a given utilization via enumeration.

In [ ]:
font = {'size'   : 13}
plt.rc('font', **font)

fig, ax1 = plt.subplots(figsize=(4.6, 2.8))

ax1.set_xlabel("# Tasks")
ax1.set_xticks(np.arange(5,max_tasks+1,5))

color = colors[0]
ax1.set_ylabel("Max LUT Size", color=color)
ax1.plot(n_tasks, n_regions, "-", color=color, linewidth=2)
ax1.set_ylim(0,np.max(n_regions)*1.05)

ax2 = ax1.twinx()

color = colors[1]
ax2.set_ylabel("Max Time ($\\mu$s)", color=color)
ax2.plot(n_tasks, elastic_times_fast/1000, "--", color=color, linewidth=2)
ax2.set_ylim(0,np.max(elastic_times_fast/1000)*1.05)

plt.tight_layout()
name = "synthetic_lookup_table"
plt.savefig(f"{name}.eps",bbox_inches='tight')
plt.savefig(f"{name}.png",bbox_inches='tight')

#### Statistics

We also report a few additional statististics.

In [ ]:

print(f"Maximum time to perform naive search over all projected harmonic intervals: {np.max(elastic_times_naive)/ms} ms")
print(f"Maximum time to generate the lookup table: {np.max(elastic_times_generate/s/60)} minutes")
print(f"Maximum time to search lookup table and assigned new periods: {np.max(elastic_times_fast)/1000} us")